In [1]:
fulldir="/Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/"
name="ppl13"

In [2]:
import cv2
from PIL import Image
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm
import numpy as np

In [3]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color=0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0],image_size[1],color

In [4]:
image=fulldir+ name+"/"
ext='.bmp'
key = "abcdefghijklm"

In [5]:
def LogisticDecryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]

    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1
    
    L_x = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L_x *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1

    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    
    C = round((L_x*L_y*10**4) % 256)
    I_prev = C
    I_prev_r = C
    I_prev_g = C
    I_prev_b = C
    I = C
    I_r = C
    I_g = C
    I_b = C
    x_prev = 4*(S_x)*(1-S_x)
    y_prev = 4*(L_x)*(1-S_y)
    x = x_prev
    y = y_prev
    imageMatrix, dw, dh, color = getImageMatrix(imageName)

    henonDecryptedImage = []
    for i in range(dw):
        row = []
        for j in range(dh):
            i_x=0
            while x <0.8 and x > 0.2 and i_x<=10:
                x = 4*x*(1-x)
                i_x=i_x+1
            i_y=0
            while y <0.8 and y > 0.2 and i_y<=10:
                y = 4*y*(1-y)
                i_y=i_y+1
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
                I_r = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_r + key_list[7]) % N) ^ imageMatrix[i][j][0]) + N-key_list[6])%N
                I_g = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_g + key_list[7]) % N) ^ imageMatrix[i][j][1]) + N-key_list[6])%N
                I_b = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_b + key_list[7]) % N) ^ imageMatrix[i][j][2]) + N-key_list[6])%N
                I_prev_r = imageMatrix[i][j][0]
                I_prev_g = imageMatrix[i][j][1]
                I_prev_b = imageMatrix[i][j][2]
                row.append((I_r,I_g,I_b))
                x = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1  
            else:
                I = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev+key_list[7]) % N) ^ imageMatrix[i][j]) + N-key_list[6])%N
                I_prev = imageMatrix[i][j]
                row.append(I)
                x = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        henonDecryptedImage.append(row)
    if color:
        im = Image.new("RGB", (dw, dh))
    else: 
        im = Image.new("L", (dw, dh)) # L is for Black and white pixels
    pix = im.load()
    for x in range(dw):
        for y in range(dh):
            pix[x, y] = henonDecryptedImage[x][y]
    print(imageName[28:-4])
    im.save(image+"decryp"+ext, "BMP")

In [6]:
def DecryptionResult(i, orig):
    #if i<10:
     #   imageName=fulldir+name+'/encrypted_frames/EncryptedResult00'+str(i)+'.bmp'
    #elif i<100:
     #   imageName=fulldir+name+'/encrypted_frames/EncryptedResult0'+str(i)+'.bmp'
    #else:
     #   imageName=fulldir+name+'/encrypted_frames/EncryptedResult'+str(i)+'.bmp'
    #im = cv2.cvtColor(np.array( Image.open(imageName)), cv2.COLOR_RGB2BGR)
    #print(type(im))
    #na = np.array(im)
    #orig = na.copy()
    na=orig #cv2.imread(imageName)
    print(na.shape)
    color_pixels= np.where(np.all(na==[255, 0, 0],axis=2))
    #print(color_pixels)
    
    zipped = np.column_stack((color_pixels[0], color_pixels[1]))

    #print(zipped)
    squares=[]
    for [y,x] in zipped:
        if np.array_equal(na[y+1,x], np.array([255, 0, 0]))==True and np.array_equal(na[y, x+1], np.array([255, 0, 0]))==True:
            #find the width and height
            w=0
            while (x+1+w)<na.shape[1] and  np.array_equal(na[y,x+1+w], np.array([255, 0, 0])):
                w=w+1
            h=0
            while (y+1+h)<na.shape[0] and np.array_equal(na[y+1+h,x], np.array([255, 0, 0])):
                h=h+1
            if w>1 and h>1:
                squares.append([x,y,w,h])
    print(squares)
    #print("frame: ", cnt)
    sum=0
    for [x,y,w,h] in squares:
        #print(a,b,c,d)
        face=na[y+1:y+h,x+1:x+w]
        if len(face)==0:
            continue
        filename=image+"decryp"+ext
        cv2.imwrite(filename, face)
        LogisticDecryption(filename, key)
        filename=image+"decryp"+ext
        DecryptResult=cv2.imread(filename)
        na[y+1:y+h,x+1:x+w]=DecryptResult
        na[y,x]=np.mean(np.array([na[y-1, x], na[y, x-1]]),axis=0) 
        na[y+1:y+h,x]=np.mean(np.array([na[y+1:y+h, x-1], na[y+1:y+h, x+1]]),axis=0) 
        na[y+h, x]=np.mean(np.array([na[y+h+1, x], na[y+h, x-1]]),axis=0)
        na[y+h, x+1:x+w]=np.mean(np.array([na[y+h-1, x+1:x+w], na[y+h+1, x+1:x+w]]),axis=0) 
        na[y+h, x+w]=np.mean(np.array([na[y+h+1, x+w], na[y+h, x+w+1]]),axis=0)
        na[y+1:y+h, x+w]=np.mean(np.array([na[y+1:y+h, x+w-1], na[y+1:y+h, x+w+1]]),axis=0) 
        na[y, x+w]=np.mean(np.array([na[y-1, x+w], na[y, x+w+1]]),axis=0) 
        na[y, x+1:x+w]=np.mean(np.array([na[y-1, x+1:x+w], na[y+1, x+1:x+w]]),axis=0) 
        diff = np.abs(orig[y+1:y+h, x+1:x+w].astype(np.float32) - na[y+1:y+h, x+1:x+w].astype(np.float32))
        squares=np.square(diff)
        sumofsquares = np.sum(squares)
        rootofsquares = sumofsquares**0.5
        loss=rootofsquares/3
        sum+=loss
        print(loss)
    #if(cnt<10): fname=fulldir+'SquareDetectionResults/decrypted_frames/decrypt0'+str(cnt)
    #else: fname=fulldir+'SquareDetectionResults/decrypted_frames/decrypt'+str(cnt)
    #cv2.imwrite(fname+ext, na)
    #cnt+=1
    sum=sum/3
    return na, sum

In [7]:
cam = cv2.VideoCapture(fulldir+name+"/results/encryptedvideo.avi")
fps = cam.get(cv2.CAP_PROP_FPS)
print(fps)
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
print(height, width)

30.0
720 1280


In [8]:
import os

files=[]
entries = os.listdir(fulldir+name+'/encrypted_frames/')

for entry in entries:
    if entry[-4:]=='.bmp':
        files.append(entry)

files.sort()

print(files)

print(len(files))

['EncryptedResult000.bmp', 'EncryptedResult001.bmp', 'EncryptedResult002.bmp', 'EncryptedResult003.bmp', 'EncryptedResult004.bmp', 'EncryptedResult005.bmp', 'EncryptedResult006.bmp', 'EncryptedResult007.bmp', 'EncryptedResult008.bmp', 'EncryptedResult009.bmp', 'EncryptedResult010.bmp', 'EncryptedResult011.bmp', 'EncryptedResult012.bmp', 'EncryptedResult013.bmp', 'EncryptedResult014.bmp', 'EncryptedResult015.bmp', 'EncryptedResult016.bmp', 'EncryptedResult017.bmp', 'EncryptedResult018.bmp', 'EncryptedResult019.bmp', 'EncryptedResult020.bmp', 'EncryptedResult021.bmp', 'EncryptedResult022.bmp', 'EncryptedResult023.bmp', 'EncryptedResult024.bmp', 'EncryptedResult025.bmp', 'EncryptedResult026.bmp', 'EncryptedResult027.bmp', 'EncryptedResult028.bmp', 'EncryptedResult029.bmp', 'EncryptedResult030.bmp', 'EncryptedResult031.bmp', 'EncryptedResult032.bmp', 'EncryptedResult033.bmp', 'EncryptedResult034.bmp', 'EncryptedResult035.bmp', 'EncryptedResult036.bmp', 'EncryptedResult037.bmp', 'EncryptedR

In [9]:
video_name = 'decryptedvideo.avi'
print(fulldir+name+"/results/")
os.chdir(fulldir+name+"/results/")
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'FFV1'), fps, (width, height))
totalloss=0
loss=0
i=0
while(True):
    ret,Img = cam.read()
    temp=Img
    #if i<10:
     #   Img= cv2.imread(fulldir+name+"/frames/00"+str(i)+".png")
    #elif i<100:
     #   Img= cv2.imread(fulldir+name+"/frames/0"+str(i)+".png")
    #else:
     #   Img= cv2.imread(fulldir+name+"/frames/"+str(i)+".png")
    if ret==False:
        break
    temploss=0
    frameout, temploss=DecryptionResult(i, Img)
    totalloss+=temploss
    if i<10:
        cv2.imwrite(fulldir+name+"/decrypted_frames/00"+str(i)+".png", frameout)
    elif i<100:
        cv2.imwrite(fulldir+name+"/decrypted_frames/0"+str(i)+".png", frameout)
    else:
        cv2.imwrite(fulldir+name+"/decrypted_frames/"+str(i)+".png", frameout)
        #if currentframe<10:
            #cv2.imwrite(fulldir+"SquareDetectionResults/encrypted_frames_bmp/EncryptedResult0"+str(currentframe)+".bmp", frame)
        #else:
            #cv2.imwrite(fulldir+"SquareDetectionResults/encrypted_frames_bmp/EncryptedResult"+str(currentframe)+".bmp", frame)
    if i<10:
        Img= cv2.imread(fulldir+name+"/frames/00"+str(i)+".png")
    elif i<100:
        Img= cv2.imread(fulldir+name+"/frames/0"+str(i)+".png")
    else:
        Img= cv2.imread(fulldir+name+"/frames/"+str(i)+".png")
    i=i+1
    loss=loss+np.mean((frameout-Img)**2)
    print("frame: ", i)
    video.write(frameout)

/Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/results/
(720, 1280, 3)
[[65, 200, 50, 88], [1020, 274, 30, 37], [939, 291, 22, 28]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  1
(720, 1280, 3)
[[66, 198, 50, 87], [1023, 270, 34, 39], [942, 291, 21, 27]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  2
(720, 1280, 3)
[[66, 199, 50, 86], [1029, 270, 33, 39], [946, 290, 23, 29]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  3
(720, 

frame:  31
(720, 1280, 3)
[[1198, 224, 65, 80], [1009, 270, 36, 46]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  32
(720, 1280, 3)
[[1213, 218, 58, 79], [1012, 269, 38, 46]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  33
(720, 1280, 3)
[[1224, 211, 51, 87], [1017, 270, 37, 48]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  34
(720, 1280, 3)
[[1021, 270, 37, 50]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  35
(720, 1280, 3)
[[1025, 273, 40, 49]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  36
(720, 1280, 3)
[[1028, 275, 43, 50]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  37

(720, 1280, 3)
[[765, 272, 29, 33], [486, 291, 29, 34]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  86
(720, 1280, 3)
[[767, 269, 29, 36], [482, 291, 29, 35]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  87
(720, 1280, 3)
[[770, 268, 29, 36], [479, 290, 30, 36], [884, 307, 25, 28]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  88
(720, 1280, 3)
[[773, 270, 30, 34], [476, 290, 31, 34], [888, 309, 23, 27]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decr

frame:  111
(720, 1280, 3)
[[869, 250, 50, 59], [683, 272, 22, 28], [445, 284, 46, 52], [963, 297, 30, 36]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  112
(720, 1280, 3)
[[879, 243, 49, 62], [682, 270, 23, 28], [441, 283, 48, 54], [964, 295, 32, 36]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  113
(720, 1280, 3)
[[888, 237, 49, 63], [683, 264, 24, 33], [436, 280, 51, 58], [968, 293, 31, 39]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing

frame:  132
(720, 1280, 3)
[[1097, 127, 100, 127], [360, 235, 80, 95], [673, 238, 32, 46], [776, 295, 28, 35]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  133
(720, 1280, 3)
[[1118, 111, 112, 136], [351, 230, 81, 97], [675, 236, 32, 46], [779, 295, 29, 34]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  134
(720, 1280, 3)
[[1151, 100, 118, 135], [339, 225, 86, 101], [677, 235, 32, 48], [785, 291, 27, 36]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secu

...

frame:  498
(720, 1280, 3)
[[592, 257, 27, 35], [100, 258, 89, 130], [787, 625, 29, 41]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  499
(720, 1280, 3)
[[593, 254, 28, 38], [81, 258, 96, 135]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  500
(720, 1280, 3)
[[593, 255, 25, 37], [63, 262, 88, 140]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  501
(720, 1280, 3)
[[592, 255, 27, 39], [35, 261, 98, 146]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  502
(720, 1280, 3)
[[591, 254, 29, 41], [9, 255, 102, 153]]


(720, 1280, 3)
[[485, 207, 47, 57], [913, 273, 31, 38]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  543
(720, 1280, 3)
[[478, 200, 49, 50], [923, 275, 27, 35]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  544
(720, 1280, 3)
[[479, 190, 48, 60], [928, 277, 31, 35]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  545
(720, 1280, 3)
[[471, 185, 53, 62]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  546
(720, 1280, 3)
[[469, 181, 53, 62]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  547
(720, 1280, 3)
[[463, 177, 53, 66]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  548
(720, 1280, 3

frame:  578
(720, 1280, 3)
[[377, 258, 37, 43], [440, 269, 28, 38], [1101, 285, 20, 29], [1132, 299, 32, 38]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  579
(720, 1280, 3)
[[375, 255, 37, 48], [438, 267, 29, 38], [1136, 300, 31, 39]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  580
(720, 1280, 3)
[[373, 253, 39, 48], [439, 266, 29, 42], [1106, 280, 22, 32], [1141, 302, 30, 39]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footag

0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  600
(720, 1280, 3)
[[221, 204, 54, 80], [328, 242, 30, 43], [396, 258, 33, 49], [956, 269, 31, 38]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  601
(720, 1280, 3)
[[214, 203, 55, 73], [324, 241, 31, 48], [391, 262, 33, 46], [431, 270, 27, 33], [957, 270, 31, 39]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-foot

frame:  615
(720, 1280, 3)
[[83, 140, 92, 115], [308, 222, 49, 72], [390, 247, 30, 37], [974, 258, 35, 43], [1245, 261, 32, 47], [1160, 278, 25, 29], [800, 279, 24, 30]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
frame:  616
(720, 1280, 3)
[[64, 131, 97, 119], [306, 223, 49, 67], [386, 246, 33, 41], [979, 260, 34, 42], [1163, 280, 24, 28]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/decryp
0.0
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl13/de

In [10]:
video.release() 
print(totalloss/len(files))
print(loss/len(files))

0.0
0.8398700444983821


In [11]:
print(i)

618


In [14]:
print(loss/(i-1))

0.8412312601296599
